In [12]:
%load_ext autoreload
%autoreload 2
import pandas as pd
pd.set_option('display.max_rows', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from absbox import mkDeal

In [14]:
from absbox import Generic

deal_data = {
    "name":"Sample with Interest Swap"
    ,"dates":{"cutoff":"2021-03-01","closing":"2021-06-15","firstPay":"2021-07-26"
     ,"payFreq":["DayOfMonth",20],"poolFreq":"MonthEnd","stated":"2030-01-01"}
    ,"pool":{'assets':[["Mortgage"
        ,{"originBalance":2200,"originRate":["fix",0.045],"originTerm":30
          ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
          ,{"currentBalance":2200
          ,"currentRate":0.08
          ,"remainTerm":20
          ,"status":"current"}]]}
    ,"accounts":{"acc01":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2020-01-03"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2020-01-03"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}}
    ,"collect":[["CollectedInterest","acc01"]
              ,["CollectedPrincipal","acc01"]
              ,["CollectedPrepayment","acc01"]
              ,["CollectedRecoveries","acc01"]]
    ,"waterfall":{"amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["settleSwap","acc01","swap1"]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
}

In [16]:
from absbox import API
localAPI = API("http://localhost:8081")

✅Connected, local lib:0.20.2, server:0.20.1

In [17]:
swap = {
    "swap1":{"settleDates":"MonthEnd"
             ,"pair":[("LPR5Y",0.01),0.05]
             ,"base":{"formula":("poolBalance",)}
             ,"start":"2021-06-25"
             ,"balance":2093.87}
}

In [21]:
deal.json

{'tag': 'MDeal',
 'contents': {'dates': {'tag': 'PreClosingDates',
   'contents': ['2021-03-01',
    '2021-06-15',
    None,
    '2030-01-01',
    ['2021-06-15', {'tag': 'MonthEnd'}],
    ['2021-07-26', {'tag': 'DayOfMonth', 'contents': 20}]]},
  'name': 'Sample with Interest Swap',
  'status': {'tag': 'Amortizing'},
  'pool': {'assets': [{'tag': 'Mortgage',
     'contents': [{'originBalance': 2200,
       'originRate': {'tag': 'Fix', 'contents': 0.045},
       'originTerm': 30,
       'period': 'Monthly',
       'startDate': '2021-02-01',
       'prinType': {'tag': 'Level'},
       'tag': 'MortgageOriginalInfo'},
      2200,
      0.08,
      20,
      None,
      {'tag': 'Current'}]}],
   'asOfDate': '2021-03-01',
   'issuanceStat': None,
   'futureCf': None},
  'bonds': {'A1': {'bndName': 'A1',
    'bndBalance': 1000,
    'bndRate': 0.07,
    'bndOriginInfo': {'originBalance': 1000,
     'originDate': '2020-01-03',
     'originRate': 0.07,
     'maturityDate': None},
    'bndInteres

In [18]:
deal = mkDeal(deal_data|{"rateSwap":swap})

In [19]:
r = localAPI.run(deal
                 ,assumptions = [{"Rate":["LPR5Y"
                                          ,["2022-01-01",0.05]
                                          ,["2023-01-01",0.06]]}]
                 ,read=True)
r['accounts']['acc01']

,balance,change,memo
date,,,
2021-06-15,0.00,0.00,<PoolInflow:CollectedRecoveries>
2021-06-15,0.00,0.00,<PoolInflow:CollectedPrepayment>
2021-06-15,0.00,0.00,<PoolInflow:CollectedPrincipal>
2021-06-15,0.00,0.00,<PoolInflow:CollectedInterest>
2021-07-26,0.00,0.00,<SeqPayFee:trusteeFee>
2021-07-26,0.88,0.88,<SettleIn:>
2021-07-26,0.88,0.00,<SettleOut:>
2021-07-26,0.00,-0.88,<PayInt:A1>
2021-07-26,0.00,0.00,<PayPrin:A1>


In [10]:
2093.87*0.01*15/365

0.8604945205479453

In [113]:
1987.34*0.01*28/365

1.5245347945205479

In [20]:
r['rateSwap']['swap1']

,balance,amount,pay,receive,due,memo
date,,,,,,
2021-06-30,2200.00,0.88,0.01,0.05,0.88,<Accure:>
2021-07-26,0.00,0.88,0.00,0.00,0.00,<SettleIn:>
2021-07-31,2200.00,7.04,0.01,0.05,7.04,<Accure:>
2021-08-20,0.00,7.04,0.00,0.00,0.00,<SettleIn:>
2021-08-31,2200.00,7.04,0.01,0.05,7.04,<Accure:>
2021-09-20,0.00,7.04,0.00,0.00,0.00,<SettleIn:>
2021-09-30,2200.00,7.04,0.01,0.05,7.04,<Accure:>
2021-10-20,0.00,7.04,0.00,0.00,0.00,<SettleIn:>
2021-10-31,2200.00,7.04,0.01,0.05,7.04,<Accure:>


In [11]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty
Date,,,,,,,,,,
2021-03-01,2200.00,0.00,0.00,0,0,0,0,0.000,None,None
2022-01-31,2093.87,106.13,8.25,0,0,0,0,0.045,None,None
2022-02-28,1987.34,106.53,7.85,0,0,0,0,0.045,None,None
2022-03-31,1880.41,106.93,7.45,0,0,0,0,0.045,None,None
2022-04-30,1773.08,107.33,7.05,0,0,0,0,0.045,None,None
2022-05-31,1665.34,107.74,6.64,0,0,0,0,0.045,None,None
2022-06-30,1557.20,108.14,6.24,0,0,0,0,0.045,None,None
2022-07-31,1448.65,108.55,5.83,0,0,0,0,0.045,None,None
2022-08-31,1339.70,108.95,5.43,0,0,0,0,0.045,None,None


In [81]:
r['pool']['flow']

,Balance,Principal,Interest,Prepayment,Default,Recovery,Loss,WAC,BorrowerNum,PrepayPenalty
Date,,,,,,,,,,
2022-01-31,2093.87,106.13,8.25,0,0,0,0,0.045,None,None
2022-02-28,1987.34,106.53,7.85,0,0,0,0,0.045,None,None
2022-03-31,1880.41,106.93,7.45,0,0,0,0,0.045,None,None
2022-04-30,1773.08,107.33,7.05,0,0,0,0,0.045,None,None
2022-05-31,1665.34,107.74,6.64,0,0,0,0,0.045,None,None
2022-06-30,1557.20,108.14,6.24,0,0,0,0,0.045,None,None
2022-07-31,1448.65,108.55,5.83,0,0,0,0,0.045,None,None
2022-08-31,1339.70,108.95,5.43,0,0,0,0,0.045,None,None
2022-09-30,1230.34,109.36,5.02,0,0,0,0,0.045,None,None
